Read in Argoverse HD map and COLMAP results

In [4]:
from transforms import affine_matrix_from_points
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from scipy.spatial import transform
import matplotlib
import matplotlib.pyplot as plt
from visualize_30hz_benchmark_data_on_map import DatasetOnMapVisualizer
from argoverse.map_representation.map_api import ArgoverseMap
import logging
import numpy as np
import numpy.linalg as LA

# path to the argoverse dataset
tracking_dataset_dir = '../argoverse-tracking/sample/'

# log_index determines which log in the argoverse dataset directory to use
log_index = 0

# load argoverse HD map
argoverse_loader = ArgoverseTrackingLoader(tracking_dataset_dir)
log_id = argoverse_loader.log_list[log_index]
argoverse_data = argoverse_loader[log_index]

am = ArgoverseMap()
city_name = argoverse_data.city_name
dataset_dir = tracking_dataset_dir
experiment_prefix = 'visualization_demo'
use_existing_files = True

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
domv = DatasetOnMapVisualizer(dataset_dir, experiment_prefix, use_existing_files=use_existing_files, log_id=argoverse_data.current_log)

# colmap_output_dir points to the results from COLMAP
colmap_output_dir = 'reconstruction/sparse/' + log_id + '/'
# detection results directory
detection_output_dir = './focal_detection/'
# where to save computed traffic sign locations
projection_output_dir = './'


# flags that determine which ring cameras to use
use_fc, use_fl, use_sl, use_rl, use_rr, use_sr, use_fr = [1, 1, 0, 0, 0, 0, 0]

#read in colmap results
poses_colmap = dict()   #dict that queries camera poses with image name
points_colmap = dict()  #dict that queries colmap feature points IDs with image name
id2name_colmap = dict() #dict that queries image name with colmap image IDs
points3D = dict()       #dict that queries colmap points' 3D coordinates with colmap point IDs

with open(colmap_output_dir + 'images.txt') as f:
    lines = f.read().splitlines()
for i,line in enumerate(lines):
    if line[0] == '#':
        continue
    else:
        if i % 2 == 0:
            fields = line.split(' ')
            image_name = fields[-1]
            image_id = int(fields[0])
            quat = fields[1:5]
            quatanion = np.array([float(q) for q in quat])
            trans = fields[5:8]
            translation = np.array([float(t) for t in trans])
            camera_id = int(fields[8])
            poses_colmap[image_name] = [quatanion, translation, image_id, camera_id]
            id2name_colmap[image_name] = image_id
        else:
            fields = line.split(' ')
            points_2d = np.array([float(pt) for pt in fields])
            points_colmap[image_name] = points_2d
            
with open(colmap_output_dir + 'points3D.txt') as f:
    lines = f.read().splitlines()
for i,line in enumerate(lines):
    if line[0] == '#':
        continue
    else: 
            fields = line.split(' ')
            points_attr = np.array([float(pt) for pt in fields])
            points3D[points_attr[0]] = np.array([points_attr[1],points_attr[2],points_attr[3]])
            

Generate the 3D point clouds for a specifc timestamp. 

In [8]:
# read in argoverse data
import numpy.linalg as LA

calib = argoverse_data.get_calibration("ring_front_center")

# Change this to the timestamp you want, idx from [0, 155]
idx = 80


city_to_egovehicle_se3 = argoverse_data.get_pose(idx)    
img_fc = argoverse_data.image_list_sync['ring_front_center'][idx]
img_fc = img_fc.split('/')
print(img_fc[-1])
image = plt.imread(tracking_dataset_dir + log_id + '/ring_front_center/' + img_fc[-1])

pc = argoverse_data.get_lidar(idx)
uv = calib.project_ego_to_image(pc).T

idx_ = np.where(np.logical_and.reduce((uv[0, :] >= 0.0, uv[0, :] < np.shape(image)[1] - 1.0,
                                                      uv[1, :] >= 0.0, uv[1, :] < np.shape(image)[0] - 1.0,
                                                      uv[2, :] > 0)))
idx_ = idx_[0]
uv1 =uv[:, idx_]
print(uv1.shape)
rgb = []
u = uv1[1, :]
print(u.shape)
v = uv1[0, :]
for i in range(uv1.shape[1]):
    rgb.append(image[int(round(u[i])),int(round(v[i])),:])
    
rgb = np.array(rgb)
print(rgb.shape)
pc = calib.project_image_to_ego(uv1.T)
pc = city_to_egovehicle_se3.transform_point_cloud(pc)
print(pc.shape)
import pandas as pd
from pyntcloud import PyntCloud
d = {'x': pc[:,0],'y': pc[:,1],'z': pc[:,2], 
             'red' : rgb[:,0], 'green' : rgb[:,1], 'blue' : rgb[:,2]}
        
cloud = PyntCloud(pd.DataFrame(data=d))
cloud.to_file("./output.ply")
import scipy.io
scipy.io.savemat('test.mat', dict(pc=pc,color=rgb))

ring_front_center_315969692021102752.jpg
(3, 17728)
(17728,)
(17728, 3)
(17728, 3)
